In [22]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from thinkbayes import Pmf, Suite

#### "A railroad numbers its locomotives in order 1...n One day you see a locomotive with the number 60. Estimate how many locomotives the railroad has"

From this statement we know the railroad has at least 60 locomotives. But how many more?    
Apply Baysian reasoning:   

1) What did we know about N before we saw the data? - Use the **Prior**

2) For any given value of N, what is the likelihood of seeing the data (a locomotive with the number 60)? - use the **Likelihood**

We don't have much on which to base a prior, but lets start with something and work from there


In [23]:
# Assume N is equally likely to be any value from 1 to 1000

hypos = range(1, 1000)

# Now Likelihood. If the have N locomotives and we are equally likely to see any of them, then the chance of seeing
# any particular one is 1 / N.

class Train(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0
        else:
            return 1.0/hypo



In [24]:
# Now to update:
suite = Train(hypos)
b = suite.Update(60)

<img src="thinkbayesLoco.png">

There are too many results to be plotted (see graph above, stolen from the book).

As you can see, the most likely value is 60. But, what are the chances that you just happened to see the train with the highest number? Not very good. 

An alternative is to compute the mean of the posterior distribution:

In [25]:
print(suite.Mean())

333.1836235027341


In [26]:
# or the long way ....
def Mean(suite):
    total = 0
    for hypo, prob in suite.Items():
        total += hypo * prob
    return total
print (Mean(suite))

333.1836235027341


In [27]:
# Use a prior with upper bound 500
hypos = range(1, 500)
suite = Train(hypos)
b = suite.Update(60)
print(suite.Mean())

206.8038772952583


In [28]:
# Use a prior with upper bound 500
hypos = range(1, 2000)
suite = Train(hypos)
b = suite.Update(60)
print(suite.Mean())

551.9730485662769


### Maybe we should rethink that PRIOR

#### We need to: *Get more data* and/or *Get more background information*

For example, we also see trains 30 and 90. We can update the distribution like this:

In [33]:
# initiate suite again
hypos = range(1, 1000) # PRIOR p(H)
suite = Train(hypos)

for data in [60, 30, 90]:
    suite.Update(data)
    
print("POSTERIOR = %d, with a PRIOR of 1000"%(suite.Mean()))
    
hypos = range(1, 500) # PRIOR p(H)
suite = Train(hypos)

for data in [60, 30, 90]:
    suite.Update(data)
    
print("POSTERIOR = %d, with a PRIOR of 500"%(suite.Mean())) # POSTERIOR of p(H) = 500 trains on the network
    
hypos = range(1, 2000) # PRIOR p(H)
suite = Train(hypos)

for data in [60, 30, 90]:
    suite.Update(data)
    
print("POSTERIOR = %d, with a PRIOR of 2000"%(suite.Mean()))    

POSTERIOR = 164, with a PRIOR of 1000
POSTERIOR = 151, with a PRIOR of 500
POSTERIOR = 171, with a PRIOR of 2000


As we add more data [60,30,90], the posterior reduces. It is not surprising give that these are all all less than the prior[500,1000,2000] that it brings the posterior down.

### An Alternative Prior

If we can't get more data, we need to improve the priors by gathering more background information. 
With some effort, we could probably find a list of companies that operate in the area, or interview an expert in rail shipping to gather information about the typical size of companies. 

Even without railroad specific economics, we could make some educated guesses. In fact, the distribution of company sizes tends to follow a power law, as reported in Science.   

This law suggests that if there are 1000 companies with fewer than 10 locomotives, there might be 100 companies with 100 locomotives, 10 companies with 1000 and possibly one company with 10 000 trains. 

This means, the number of companies with a given size is inversely proportional to size:

\begin{equation} PMF(x) \:\alpha \left(\frac{1}{x}\right)^\alpha \end{equation}

where $PMF(x)$ is the probability mass function of $x$ and $\alpha$ is a parameter that is ofter near 1.

We can construct a power law prior like this:  

In [35]:
# Taken from the first "Estimation" tutorial
class Dice(Suite): 
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0 
        else:
            return 1.0/hypo

# The likelihood function is the same in the Train as the Dice
class Train(Dice):
    def __init__(self, hypos, alpha = 1.0):  # Adding alpha to the arguments
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, hypo**(-alpha))  # adding in the power law here to alter the prior
        self.Normalize()


In [36]:
# Code to construct the prior
hypos = range(1,1001)
suite = Train(hypos)

Again the upper bound is arbitrary, but with a power law prior, the posterior is less sensitive to this choice. 

Using the background information represented in the power law prior, we can all but eliminate values of N > 700.

<img src="thinkbayesLoco2.png">

In [38]:
# Some number for the above plot. Using the power law as a prior.
# initiate suite again
hypos = range(1, 1001) # PRIOR p(H)
suite = Train(hypos)

for data in [60, 30, 90]:
    suite.Update(data)
    
print("POSTERIOR = %d, with a PRIOR of 1000"%(suite.Mean()))
    
hypos = range(1, 501) # PRIOR p(H)
suite = Train(hypos)

for data in [60, 30, 90]:
    suite.Update(data)
    
print("POSTERIOR = %d, with a PRIOR of 500"%(suite.Mean())) # POSTERIOR of p(H) = 500 trains on the network
    
hypos = range(1, 2001) # PRIOR p(H)
suite = Train(hypos)

for data in [60, 30, 90]:
    suite.Update(data)
    
print("POSTERIOR = %d, with a PRIOR of 2000"%(suite.Mean()))  

POSTERIOR = 133, with a PRIOR of 1000
POSTERIOR = 130, with a PRIOR of 500
POSTERIOR = 133, with a PRIOR of 2000


Now the differences are much smaller. Adding the power law to the prior makes it more realistic as it is based on general information about the size of companies and it behaves better in practice. Basically the more companies there are, the fewer trains they are likely to have. 

I am not sure what the size of companies has to do with the number of trains in our prior however. We don't have any information of how many companies operate in one area. 